In [45]:
import sys
import gi
gi.require_version('Vips', '8.0')
from gi.repository import Vips
import math

In [46]:
SHAVE = 0 # calculate a shave pixel number from this shave proportion like shavepix = im.width * SHAVE / 2
CSCALE = 10
CPERCENT = 99.9
CTHRESHOLD = 10
BLACK = 2 * 256
WHITE = 170 * 256

In [47]:
infile = "/Users/mah/Temp/neg-nb.tiff"
ims = [Vips.Image.new_from_file(infile)]

In [48]:
def shrink(im, x, y):
    xscale = 1
    yscale = 1
    
    if x >= 1:
        xscale = x / im.width
    else:
        if x < 1 and x > 0:
            xscale = x
            
    if y >= 1:
        yscale = y / im.height
    else:
        if y < 1 and y > 0:
            yscale = y

    return im.resize(xscale, vscale = yscale)

In [49]:
def vtrim(im, levels, lt = False):

    
    shave = int(im.Yres*2/3)    
    stripe = shrink(im, 1, 0)
    
    mask = Vips.Image.black(stripe.width, stripe.height, bands = 1)
    mask = mask.linear([0], [255])
    
    for i in range(im.Bands):
        if lt:
            mask = mask & (stripe[i] > levels[i]) 
        else:
            mask = mask & (stripe[i] < levels[i])

    for i in range(int(mask.height / 3)):
        if mask(0, i)[0] > 0:
            break

    for j in range(int(mask.height / 3)):
        if mask(0, mask.height - j - 1)[0] > 0:
            break

    if j > 0:
        j += shave
       
    if i > 0:
        i += shave

    j = mask.height - j - i 

    im = im.extract_area(0, i, im.width, j)             
        
    return im

In [50]:
def htrim(im, levels, lt = False):
    shave = int(im.Xres*2/3)    
    stripe = shrink(im, 0, 1)
    
    mask = Vips.Image.black(stripe.width, stripe.height, bands = 1)
    mask = mask.linear([0], [255])
    
    for i in range(im.Bands):
        if lt:
            mask = mask & (stripe[i] > levels[i]) 
        else:
            mask = mask & (stripe[i] < levels[i])

    for i in range(int(mask.width / 3)):
        if mask(i, 0)[0] > 0:
            break

    for j in range(int(mask.width / 3)):
        if mask(mask.width - j - 1, 0)[0] > 0:
            break

    if j > 0:
        j += shave
       
    if i > 0:
        i += shave

    j = mask.width - j - i 

    im = im.extract_area(i, 0, j, im.height)            
    
    return im

In [51]:
def prune(im):
    im = vtrim(im, [BLACK for b in range(im.Bands)], lt = True)
    im = htrim(im, [BLACK for b in range(im.Bands)], lt = True)
    im = vtrim(im, [WHITE for b in range(im.Bands)])
    im = htrim(im, [WHITE for b in range(im.Bands)])
    #im = trim(im, [base for b in range(im.Bands)], horizontal = True)
    #print(im.width, im.height)
    return im

In [52]:
def filmbase(im):
    return [int(band.percent(CPERCENT) - band.deviate() / 2) for band in im.bandsplit()]

In [53]:
def split(im):
    regions = []
    crops = []
    regionstart = None

    base = filmbase(im)
    stripe = shrink(im, 1, 0)
    mask = ((stripe[0] < base[0]) & (stripe[1] < base[1]) & (stripe[2] < base[2])) 
    
    #mask.write_to_file('/Users/mah/Temp/out/mask.tiff')
    mask = shrink(mask, 0, 1 / CSCALE)
    
    for j in range(mask.height):
        if (mask(0, j)[0] == 0) or (j == mask.height):
            if regionstart:
                if j - regionstart >= (0.7 * im.width / CSCALE):
                    regions.append((regionstart * CSCALE, min((j - regionstart) * CSCALE, im.height)))
                regionstart = None
        else:        
            if not(regionstart):
                regionstart = j
    for (y, l) in regions:
        crops.append(im.extract_area(0, y, im.width, l))
    return crops

In [54]:
def getframes(ims):
    frames = []
    for im in ims:
        im = prune(im)
        frames += split(im)
    return frames

In [55]:
frames = getframes(ims)
for f in range(len(frames)):
    frames[f].write_to_file('/Users/mah/Temp/out/test_' + str(f) + '.tiff')

TODO: Identifying rows containing pure lack or pure white (no film or scanner holder): 

Take the middle one-third of the image and calculate the minimum and the maximum. Now if in the first or last rows we find averages below the minimum or above the maximum from the middle one-third, we get rid of them!

TODO: Rotate the image and run the same cut process to trim the empty negative area on the sides